*In our dev this file was called "train_best_lstm-2.ipynb". I modified it slightly for the cantemist-ner repo!*

In [ ]:
!pip install -U biome 

In [8]:
from biome.text import Pipeline
from biome.text.configuration import VocabularyConfiguration, TrainerConfiguration
from biome.text.data import DataSource
from biome.text.loggers import MlflowLogger
from pathlib import Path
import pandas as pd
import allennlp

In [9]:
train_path = Path("data/NER/train_for_bert.json")
dev1_path = Path("data/NER/dev1_for_bert.json")
dev2_path = Path("data/NER/dev2_for_bert.json")
train_full_path = Path("data/NER/train_full.json")

# Create train_full data set

In [10]:
train_full = pd.concat([pd.read_json(train_path, orient="records", lines=True), 
                        pd.read_json(dev1_path, orient="records", lines=True),
                        pd.read_json(dev2_path, orient="records", lines=True)], 
                       ignore_index=True)

In [11]:
train_full.to_json(train_full_path, orient="records", lines=True)

# Define data sources

In [3]:
train_ds = DataSource(str(train_full_path), orient="records", lines=True)
valid_ds = DataSource(str(dev2_path), orient="records", lines=True)

In [18]:
train_ds.to_dataframe().compute()

,text_org,text,labels,file,sentence_offset,entity_text,path
0,-TAC abdominal urgente: En lóbulo hepático izq...,"[-, TAC, abdominal, urgente, :, En, lóbulo, he...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cc_onco340.txt,976,,data/NER_david/train_full.json
1,"Se efectua una RM de control a los 10 días, ap...","[Se, efectua, una, RM, de, control, a, los, 10...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cc_onco1175.txt,8309,[],data/NER_david/train_full.json
2,"Anti-Gangliósidos IgG e IgM Negativo, IgG séri...","[Anti, -, Gangliósidos, IgG, e, IgM, Negativo,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cc_onco570.txt,3696,,data/NER_david/train_full.json
3,Material constituido por necrosis y grupos ais...,"[Material, constituido, por, necrosis, y, grup...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cc_onco1333.txt,1521,[],data/NER_david/train_full.json
4,La paciente niega episodios previos similares.,"[La, paciente, niega, episodios, previos, simi...","[O, O, O, O, O, O, O]",cc_onco427.txt,1004,,data/NER_david/train_full.json
...,...,...,...,...,...,...,...
35872,"Se identifican numerosas figuras mitóticas, al...","[Se, identifican, numerosas, figuras, mitótica...","[O, O, O, O, O, O, O, O, O, O, O]",cc_onco1488.txt,2894,[],data/NER_david/train_full.json
35873,"» Dados los hallazgos, se comenta el caso en C...","[», Dados, los, hallazgos, ,, se, comenta, el,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",cc_onco193.txt,2154,,data/NER_david/train_full.json
35874,Se diagnostica de úlcera neurotrófica OI como ...,"[Se, diagnostica, de, úlcera, neurotrófica, OI...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",cc_onco815.txt,5090,,data/NER_david/train_full.json
35875,Todo ello era compatible preliminarmente con u...,"[Todo, ello, era, compatible, preliminarmente,...","[O, O, O, O, O, O, O, B-MORFOLOGIA_NEOPLASIA, ...",cc_onco1077.txt,6265,"[SE, SE, SE atípico, SE]",data/NER_david/train_full.json


# Define pipeline

In [4]:
#pl_trained1 = Pipeline.from_pretrained("runs/tune/first-experiment/tune_hpo_train_14_hidden_size=128,num_layers=1,type=lstm,dropout=0.16517,hidden_size=128,dropout=0.26896,lr=0.0038931_2020-07-24_22-55-05k2vwybhk/training/model.tar.gz")
#pipeline_config = pl_trained1.config.as_dict()

In [4]:
pipeline_config = {'name': 'candemist-ner-first-hpo',
 'tokenizer': {'lang': 'en',
  'max_sequence_length': None,
  'max_nr_of_sentences': None,
  'text_cleaning': None,
  'segment_sentences': False,
  'start_tokens': None,
  'end_tokens': None},
 'features': {'word': {'embedding_dim': 300,
   'lowercase_tokens': False,
   'trainable': True,
   'weights_file': None},
  'char': {'embedding_dim': 64,
   'lowercase_characters': False,
   'encoder': {'bidirectional': True,
    'hidden_size': 128,
    'num_layers': 1,
    'type': 'gru'},
   'dropout': 0.16517050992687604},
  'transformers': None},
 'head': {'dropout': 0.2689579604286324,
  'feedforward': {'activations': ['relu'],
   'dropout': [0],
   'hidden_dims': [32],
   'num_layers': 1},
  'labels': ['O',
   'I-MORFOLOGIA_NEOPLASIA',
   'U-MORFOLOGIA_NEOPLASIA',
   'B-MORFOLOGIA_NEOPLASIA',
   'L-MORFOLOGIA_NEOPLASIA'],
  'type': 'biome.text.modules.heads.token_classification.TokenClassification'},
 'encoder': {'bidirectional': True,
  'hidden_size': 128,
  'input_size': 556,
  'num_layers': 1,
  'type': 'allennlp.modules.seq2seq_encoders.pytorch_seq2seq_wrapper.LstmSeq2SeqEncoder'}}

In [3]:
pipeline_config

{'name': 'candemist-ner-first-hpo',
 'tokenizer': {'lang': 'en',
  'max_sequence_length': None,
  'max_nr_of_sentences': None,
  'text_cleaning': None,
  'segment_sentences': False,
  'start_tokens': None,
  'end_tokens': None},
 'features': {'word': {'embedding_dim': 300,
   'lowercase_tokens': False,
   'trainable': True,
   'weights_file': None},
  'char': {'embedding_dim': 64,
   'lowercase_characters': False,
   'encoder': {'bidirectional': True,
    'hidden_size': 128,
    'num_layers': 1,
    'type': 'gru'},
   'dropout': 0.16517050992687604},
  'transformers': None},
 'head': {'dropout': 0.2689579604286324,
  'feedforward': {'activations': ['relu'],
   'dropout': [0],
   'hidden_dims': [32],
   'num_layers': 1},
  'labels': ['O',
   'I-MORFOLOGIA_NEOPLASIA',
   'U-MORFOLOGIA_NEOPLASIA',
   'B-MORFOLOGIA_NEOPLASIA',
   'L-MORFOLOGIA_NEOPLASIA'],
  'type': 'biome.text.modules.heads.token_classification.TokenClassification'},
 'encoder': {'bidirectional': True,
  'hidden_size': 12

In [7]:
pipeline_config["features"]["word"]["weights_file"] = "cc.es.300.vec.gz"

In [8]:
pipeline_config["encoder"]["hidden_size"] = 512

In [9]:
pipeline_config["features"]["word"]["lowercase_tokens"] = True

In [10]:
pipeline_config["features"]["char"]["lowercase_characters"] = True

In [11]:
pipeline_config["head"]["feedforward"] = None

In [12]:
pl = Pipeline.from_config(pipeline_config) ; pl._model

PipelineModel(
  (_head): TokenClassification(
    (backbone): ModelBackbone(
      (embedder): BasicTextFieldEmbedder(
        (token_embedder_word): Embedding()
        (token_embedder_char): TokenCharactersEncoder(
          (_embedding): TimeDistributed(
            (_module): Embedding()
          )
          (_encoder): TimeDistributed(
            (_module): GruSeq2VecEncoder(
              (_module): GRU(64, 128, batch_first=True, bidirectional=True)
            )
          )
          (_dropout): Dropout(p=0.16517050992687604, inplace=False)
        )
      )
      (encoder): LstmSeq2SeqEncoder(
        (_module): LSTM(556, 512, batch_first=True, bidirectional=True)
      )
    )
    (dropout): Dropout(p=0.2689579604286324, inplace=False)
    (_label_projection_layer): TimeDistributed(
      (_module): Linear(in_features=1024, out_features=5, bias=True)
    )
    (_crf): ConditionalRandomField()
  )
)

In [13]:
vocab_config = VocabularyConfiguration(
    sources=[train_ds], min_count={"word": 2}
)

In [14]:
pl.create_vocabulary(vocab_config)

# Define trainer and logger

In [15]:
trainer_dict={
    "optimizer": {
        "type": "adamw",
        "lr": 0.0038931174186587806,
        "weight_decay": 0.01,
    },
    "learning_rate_scheduler": {
      "type": "step",
      "step_size":2, 
      "gamma":0.1
    },
    "batch_size": 32,
    "num_epochs": 4,
    "validation_metric": "+f1-measure-overall",
    "patience":3
}
trainer_config = TrainerConfiguration(**trainer_dict)

In [16]:
mlflow_logger = MlflowLogger(experiment_name="cantemist-ner", run_name="best_normal_lstm_full_train")

# Execute training

In [17]:
pl.train(
    output="runs/best_normal_lstm/lstm_full_train",
    training=train_ds,
    #validation=valid_ds,
    trainer=trainer_config,
    loggers=[mlflow_logger],
)

2020-08-02 16:48:27,111 - allennlp.common.params - INFO - config = <biome.text.configuration.PipelineConfiguration object at 0x7f450684d0d0>
2020-08-02 16:48:27,372 - allennlp.common.params - INFO - type = single_id
2020-08-02 16:48:27,373 - allennlp.common.params - INFO - namespace = word
2020-08-02 16:48:27,374 - allennlp.common.params - INFO - lowercase_tokens = True
2020-08-02 16:48:27,375 - allennlp.common.params - INFO - start_tokens = None
2020-08-02 16:48:27,375 - allennlp.common.params - INFO - end_tokens = None
2020-08-02 16:48:27,376 - allennlp.common.params - INFO - feature_name = text
2020-08-02 16:48:27,377 - allennlp.common.params - INFO - default_value = THIS IS A REALLY UNLIKELY VALUE THAT HAS TO BE A STRING
2020-08-02 16:48:27,378 - allennlp.common.params - INFO - token_min_padding_length = 0
2020-08-02 16:48:27,379 - allennlp.common.params - INFO - type = characters
2020-08-02 16:48:27,381 - allennlp.common.params - INFO - namespace = char
2020-08-02 16:48:27,382 - a

2020-08-02 16:49:11,503 - allennlp.modules.token_embedders.embedding - INFO - Initializing pre-trained embedding layer


2020-08-02 16:49:11,705 - allennlp.modules.token_embedders.embedding - INFO - Pretrained embeddings were found for 11990 out of 14133 tokens
2020-08-02 16:49:11,712 - allennlp.common.params - INFO - token_embedders.char.type = character_encoding
2020-08-02 16:49:11,716 - allennlp.common.params - INFO - token_embedders.char.embedding.embedding_dim = 64
2020-08-02 16:49:11,716 - allennlp.common.params - INFO - token_embedders.char.embedding.num_embeddings = None
2020-08-02 16:49:11,717 - allennlp.common.params - INFO - token_embedders.char.embedding.projection_dim = None
2020-08-02 16:49:11,718 - allennlp.common.params - INFO - token_embedders.char.embedding.weight = None
2020-08-02 16:49:11,719 - allennlp.common.params - INFO - token_embedders.char.embedding.padding_index = None
2020-08-02 16:49:11,720 - allennlp.common.params - INFO - token_embedders.char.embedding.trainable = True
2020-08-02 16:49:11,720 - allennlp.common.params - INFO - token_embedders.char.embedding.max_norm = None


2020-08-02 17:17:10,553 - allennlp.training.tensorboard_writer - INFO -                                     Training |  Validation
2020-08-02 17:17:10,557 - allennlp.training.tensorboard_writer - INFO - accuracy                        |     0.984  |       N/A
2020-08-02 17:17:10,559 - allennlp.training.tensorboard_writer - INFO - accuracy_1                      |     0.984  |       N/A
2020-08-02 17:17:10,562 - allennlp.training.tensorboard_writer - INFO - f1-measure-MORFOLOGIA_NEOPLASIA |     0.724  |       N/A
2020-08-02 17:17:10,563 - allennlp.training.tensorboard_writer - INFO - f1-measure-overall              |     0.724  |       N/A
2020-08-02 17:17:10,566 - allennlp.training.tensorboard_writer - INFO - loss                            |    31.405  |       N/A
2020-08-02 17:17:10,567 - allennlp.training.tensorboard_writer - INFO - precision-MORFOLOGIA_NEOPLASIA  |     0.756  |       N/A
2020-08-02 17:17:10,569 - allennlp.training.tensorboard_writer - INFO - precision-overall      

2020-08-02 17:17:11,049 - allennlp.training.trainer - INFO - Epoch duration: 0:27:49.458116
2020-08-02 17:17:11,051 - allennlp.training.trainer - INFO - Estimated training time remaining: 1:23:28
2020-08-02 17:17:11,054 - allennlp.training.trainer - INFO - Epoch 1/3
2020-08-02 17:17:11,054 - allennlp.training.trainer - INFO - Worker 0 memory usage MB: 1581.404
2020-08-02 17:17:11,108 - allennlp.training.trainer - INFO - Training


...

2020-08-02 18:11:20,105 - allennlp.training.tensorboard_writer - INFO -                                     Training |  Validation
2020-08-02 18:11:20,107 - allennlp.training.tensorboard_writer - INFO - accuracy                        |     0.994  |       N/A
2020-08-02 18:11:20,110 - allennlp.training.tensorboard_writer - INFO - accuracy_1                      |     0.994  |       N/A
2020-08-02 18:11:20,113 - allennlp.training.tensorboard_writer - INFO - f1-measure-MORFOLOGIA_NEOPLASIA |     0.907  |       N/A
2020-08-02 18:11:20,114 - allennlp.training.tensorboard_writer - INFO - f1-measure-overall              |     0.907  |       N/A
2020-08-02 18:11:20,116 - allennlp.training.tensorboard_writer - INFO - loss                            |     5.085  |       N/A
2020-08-02 18:11:20,117 - allennlp.training.tensorboard_writer - INFO - precision-MORFOLOGIA_NEOPLASIA  |     0.910  |       N/A
2020-08-02 18:11:20,119 - allennlp.training.tensorboard_writer - INFO - precision-overall      

2020-08-02 18:11:20,669 - allennlp.training.trainer - INFO - Epoch duration: 0:27:09.476878
2020-08-02 18:11:20,670 - allennlp.training.trainer - INFO - Estimated training time remaining: 0:27:19
2020-08-02 18:11:20,673 - allennlp.training.trainer - INFO - Epoch 3/3
2020-08-02 18:11:20,674 - allennlp.training.trainer - INFO - Worker 0 memory usage MB: 1807.144
2020-08-02 18:11:20,740 - allennlp.training.trainer - INFO - Training


2020-08-02 18:38:24,587 - allennlp.training.tensorboard_writer - INFO -                                     Training |  Validation
2020-08-02 18:38:24,588 - allennlp.training.tensorboard_writer - INFO - accuracy                        |     0.995  |       N/A
2020-08-02 18:38:24,590 - allennlp.training.tensorboard_writer - INFO - accuracy_1                      |     0.995  |       N/A
2020-08-02 18:38:24,592 - allennlp.training.tensorboard_writer - INFO - f1-measure-MORFOLOGIA_NEOPLASIA |     0.927  |       N/A
2020-08-02 18:38:24,594 - allennlp.training.tensorboard_writer - INFO - f1-measure-overall              |     0.927  |       N/A
2020-08-02 18:38:24,595 - allennlp.training.tensorboard_writer - INFO - loss                            |     3.938  |       N/A
2020-08-02 18:38:24,596 - allennlp.training.tensorboard_writer - INFO - precision-MORFOLOGIA_NEOPLASIA  |     0.928  |       N/A
2020-08-02 18:38:24,598 - allennlp.training.tensorboard_writer - INFO - precision-overall      

2020-08-02 18:38:25,177 - allennlp.training.trainer - INFO - Epoch duration: 0:27:04.504302
2020-08-02 18:38:25,182 - allennlp.training.checkpointer - INFO - loading best weights
2020-08-02 18:38:25,204 - allennlp.models.archival - INFO - archiving weights and vocabulary to runs/best_normal_lstm/lstm_full_train/model.tar.gz
Registered model 'candemist-ner-first-hpo' already exists. Creating a new version of this model...
Created version '31' of model 'candemist-ner-first-hpo'.


TrainingResults(model_path='runs/best_normal_lstm/lstm_full_train/model.tar.gz', metrics={'best_epoch': 3, 'peak_worker_0_memory_MB': 1807.144, 'training_duration': '1:49:03.014038', 'training_start_epoch': 0, 'training_epochs': 3, 'epoch': 3, 'training_accuracy': 0.995463095028384, 'training_accuracy_1': 0.995463095028384, 'training_precision-MORFOLOGIA_NEOPLASIA': 0.9279664501274566, 'training_recall-MORFOLOGIA_NEOPLASIA': 0.9254551418730523, 'training_f1-measure-MORFOLOGIA_NEOPLASIA': 0.9267090946417074, 'training_precision-overall': 0.9279664501274566, 'training_recall-overall': 0.9254551418730523, 'training_f1-measure-overall': 0.9267090946417074, 'training_loss': 3.9377396170468253, 'training_reg_loss': 0.0, 'training_worker_0_memory_MB': 1807.144})